# AI기법과 활용 - Week 06
Transfer Learning
- https://lcyking.tistory.com/78
- https://github.com/SeHwanJoo/cifar10-vgg16/blob/master/vgg16.py
____

## 1. W&B 세팅

In [ ]:
import sys
!{sys.executable} -m pip install wandb
!pip install wandb

import wandb



## 2. 데이터셋 세팅 (CIFAR-10)

In [ ]:
from tensorflow.keras.datasets import cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

In [ ]:
import matplotlib.pyplot as plt
class_names = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')
plt.figure(figsize=(10,10))
for i in range(25): # 25 images
    plt.subplot(5,5,i+1) # matrix of 5 X 5 array
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary) # printing binary/black and white image
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel("%s %s" % (train_labels[i], class_names[train_labels[i][0]])) # Assigning name to each image
plt.show()

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

train_oh_labels = to_categorical(train_labels)
test_oh_labels = to_categorical(test_labels)

## 훈련 데이터 셋을 5천개만 사용할 경우
tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=0.9)

print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

## 3. VGG16 - No Pretrained Weights

In [ ]:
# https://github.com/SeHwanJoo/cifar10-vgg16/blob/master/vgg16.py

import tensorflow as tf
from tensorflow import keras
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv16 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=256,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dense2 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.conv2(net)
        net = self.maxPooling1(net)
        net = self.conv3(net)
        net = self.conv4(net)
        net = self.maxPooling2(net)
        net = self.conv5(net)
        net = self.conv6(net)
        net = self.conv7(net)
        net = self.maxPooling3(net)
        net = self.conv11(net)
        net = self.conv12(net)
        net = self.conv13(net)
        net = self.maxPooling5(net)
        net = self.conv14(net)
        net = self.conv15(net)
        net = self.conv16(net)
        net = self.maxPooling6(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.batchnorm(net)
        net = self.dropOut(net)
        net = self.dense2(net)
        net = self.softmax(net)
        return net


In [ ]:
model = VGG16Model()
model.build((5000,32,32,3))
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from wandb.keras import WandbCallback
run = wandb.init(project="image-classification")

train_gen = ImageDataGenerator(
    horizontal_flip=True,
    rescale=1/255.0
)
# 검증용 데이터 세트는 scale만 바꾸어주고 augmentation 적용하면 안됨
val_gen = ImageDataGenerator(rescale=1/255.0)
test_gen = ImageDataGenerator(rescale=1/255.0)

flow_tr_gen = train_gen.flow(tr_images, tr_oh_labels, batch_size=64, shuffle=True)
flow_val_gen = val_gen.flow(val_images, val_oh_labels, batch_size=64, shuffle=False)
flow_test_gen = test_gen.flow(test_images, test_oh_labels, batch_size=64, shuffle=False)

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# 3번 반복내에 validation loss가 줄어들지 않으면 learning rate를 0.2 감소
lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
# 5번 반복내에 validation loss가 줄어들지 않으면 강제종료
st_cb = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

result = model.fit(flow_tr_gen, epochs=50, validation_data=flow_val_gen, callbacks=[lr_cb, st_cb, WandbCallback()])


model.evaluate(flow_test_gen)
run.finish()

## 4. VGG16 - Pretrained Weights

In [ ]:
from tensorflow.keras.applications import VGG16
vgg_base = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))


In [ ]:
vgg_base.summary()

In [ ]:



from tensorflow.keras import models, Input, Model
from tensorflow.keras import layers
model = models.Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax', name='output'))


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from wandb.keras import WandbCallback
run = wandb.init(project="image-classification")

train_gen = ImageDataGenerator(
    horizontal_flip=True,
    rescale=1/255.0
)
# 검증용 데이터 세트는 scale만 바꾸어주고 augmentation 적용하면 안됨
val_gen = ImageDataGenerator(rescale=1/255.0)
test_gen = ImageDataGenerator(rescale=1/255.0)

flow_tr_gen = train_gen.flow(tr_images, tr_oh_labels, batch_size=64, shuffle=True)
flow_val_gen = val_gen.flow(val_images, val_oh_labels, batch_size=64, shuffle=False)
flow_test_gen = test_gen.flow(test_images, test_oh_labels, batch_size=64, shuffle=False)

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# 3번 반복내에 validation loss가 줄어들지 않으면 learning rate를 0.2 감소
lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
# 5번 반복내에 validation loss가 줄어들지 않으면 강제종료
st_cb = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

result = model.fit(flow_tr_gen, epochs=50, validation_data=flow_val_gen, callbacks=[lr_cb, st_cb, WandbCallback()])


model.evaluate(flow_test_gen)
run.finish()